In [ ]:
import pandas as pd 
import yfinance as yf 
import numpy as np 
import datetime as dt 

In [122]:
df = pd.read_csv('1_sorted_tsla.csv')
# df[abs(df['strike_price'] - 100) < 120]

,date,exdate,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,...,current_price,time_to_maturity,historical_volatility,BS_price,avg_bid_offer,bspricediff,real_iv,r,SP/CP,predicted_iv
0,2018-08-07,2019-01-18,1,10.00,0.05,0.06,354,4623,1.950605,-0.000294,...,379.57,0.449315,0.567799,8.629651e-22,0.055,0.055000,1.964774,0.045,0.026346,1.799794
1,2018-08-08,2019-01-18,1,10.00,0.06,0.07,2201,4959,1.982271,-0.000344,...,370.34,0.446575,0.569119,1.501936e-21,0.065,0.065000,1.994522,0.045,0.027002,1.794803
2,2018-08-13,2019-01-18,1,10.00,0.07,0.09,2203,11677,2.041523,-0.000424,...,356.41,0.432877,0.563654,3.675879e-22,0.080,0.080000,2.054195,0.045,0.028058,1.799548
3,2018-08-10,2019-01-18,0,10.00,343.80,348.50,0,0,3.360902,0.995591,...,355.49,0.441096,0.569195,3.456865e+02,346.150,0.463464,2.553084,0.045,0.028130,2.187002
4,2018-08-10,2019-01-18,1,10.00,0.09,0.10,639,11092,2.060635,-0.000485,...,355.49,0.441096,0.569195,2.404377e-21,0.095,0.095000,2.072838,0.045,0.028130,1.787735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4539506,2023-08-18,2024-06-21,1,216.67,34.70,35.05,6,5618,0.496607,-0.392539,...,215.49,0.843836,0.612086,4.357434e+01,34.875,-8.699341,0.496184,0.045,1.005476,0.670357
4539685,2023-08-18,2023-08-25,0,217.50,4.85,4.95,29556,956,0.481604,0.463539,...,215.49,0.019178,0.241996,2.072745e+00,4.900,2.827255,0.482327,0.045,1.009328,0.649982
4539686,2023-08-18,2023-08-25,1,217.50,6.35,6.45,15252,2659,0.454849,-0.541730,...,215.49,0.019178,0.241996,3.895121e+00,6.400,2.504879,0.455120,0.045,1.009328,0.655111
4539687,2023-08-18,2023-09-01,0,217.50,7.30,7.40,3485,336,0.480818,0.487559,...,215.49,0.038356,0.224190,3.027146e+00,7.350,4.322854,0.481904,0.045,1.009328,0.653929


In [134]:
filtered_df = df[abs(df['time_to_maturity'] - 30/365) < 3e-2]

# We need to keep option's date, exdate, cp_flag, strike_price, impl_volatility, time_to_maturity
filtered_df = filtered_df[['date', 'exdate', 'cp_flag', 'delta', 'impl_volatility', 'time_to_maturity']]
filtered_df['date'] = pd.to_datetime(filtered_df['date'])
filtered_df['exdate'] = pd.to_datetime(filtered_df['exdate'])
filtered_df = filtered_df[abs(abs(filtered_df['delta']) - 0.5) <= 0.05]
filtered_df.sort_values(by='date', ascending=False)

# test_df = filtered_df.copy()
# test_df.drop_duplicates(subset=['date'], inplace=True)
# test_df = test_df.sort_values(by='date', ascending=False)
# test_df


,date,exdate,cp_flag,delta,impl_volatility,time_to_maturity
4540611,2023-08-31,2023-10-06,1,-0.531696,0.477869,0.098630
4539566,2023-08-31,2023-10-06,1,-0.480169,0.479928,0.098630
4539565,2023-08-31,2023-10-06,0,0.524161,0.484730,0.098630
4539564,2023-08-31,2023-09-29,1,-0.489367,0.446091,0.079452
4539563,2023-08-31,2023-09-29,0,0.514596,0.448506,0.079452
...,...,...,...,...,...,...
47889,2018-08-06,2018-09-14,1,-0.453126,0.433208,0.106849
47888,2018-08-06,2018-09-14,0,0.545787,0.432006,0.106849
47887,2018-08-06,2018-09-07,1,-0.453883,0.427940,0.087671
47886,2018-08-06,2018-09-07,0,0.545691,0.426261,0.087671


In [136]:
# Filter so that each date only has one option, chosen by abs(delta) being closest to 0.5
filtered_df = (
    filtered_df
    .assign(delta_abs=filtered_df["delta"].abs(),
            dist=(filtered_df["delta"].abs() - 0.5).abs())
    .sort_values(["date", "dist"])
    .drop_duplicates("date", keep="first")
    .drop(columns=["delta_abs", "dist"])
)
filtered_df

,date,exdate,cp_flag,delta,impl_volatility,time_to_maturity
49343,2018-08-06,2018-09-07,1,-0.500416,0.422765,0.087671
48631,2018-08-07,2018-08-31,0,0.502370,0.453020,0.065753
49072,2018-08-08,2018-08-31,0,0.499698,0.474713,0.063014
49844,2018-08-09,2018-09-14,1,-0.501533,0.530763,0.098630
49664,2018-08-10,2018-09-14,0,0.500920,0.506580,0.095890
...,...,...,...,...,...,...
4539522,2023-08-25,2023-09-15,1,-0.493554,0.437119,0.057534
4539454,2023-08-28,2023-09-22,1,-0.484767,0.449784,0.068493
4539766,2023-08-29,2023-09-29,1,-0.499308,0.445062,0.084932
4539845,2023-08-30,2023-09-29,0,0.501849,0.452296,0.082192


In [131]:
tsla_df = yf.download('TSLA', start='2018-08-06', end='2023-09-01') 
tsla_df.index = pd.to_datetime(tsla_df.index)
tsla_df = tsla_df.sort_index(ascending=False)
tsla_df


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,TSLA,TSLA,TSLA,TSLA,TSLA
Date,,,,,
2023-08-31,258.079987,261.179993,255.050003,255.979996,108861700
2023-08-30,256.899994,260.510010,250.589996,254.199997,121988400
2023-08-29,257.179993,257.480011,237.770004,238.580002,134047600
2023-08-28,238.820007,244.380005,235.350006,242.580002,107673700
2023-08-25,238.589996,239.000000,230.350006,231.309998,106612200
...,...,...,...,...,...
2018-08-10,23.699333,24.000000,23.066668,23.600000,173280000
2018-08-09,23.496668,24.467333,23.048668,24.370001,257757000


In [ ]:
close_df = tsla_df['Close']
close_df.columns = ['Close']
close_df = close_df.reset_index().rename(columns={"Date": "date"})

merged_df = pd.merge(filtered_df, close_df, on='date', how='left')
merged_df

# Get 30 days implied volatility 
merged_df['Log_return'] = np.log(close_df['Close'] / close_df['Close'].shift(1))

def get_RV(df):
    time_to_maturity = int(df['exdate'] - df['date'])
    return df['Log_return'].rolling(time_to_maturity).std() * np.sqrt(252)

merged_df["RV"] = get_RV(merged_df)
merged_df 

ValueError: window must be an integer 0 or greater

In [99]:
close_df.columns


Index(['date', 'Close', 'Log_return', '30_RV'], dtype='object')

In [106]:
# Merge close_df and df on date 
merged_df = pd.merge(df, close_df, on='date', how='left')


In [107]:
merged_df

,date,exdate,cp_flag,strike_price,impl_volatility,time_to_maturity,Close,Log_return,30_RV
0,2018-09-19,2018-10-19,0,25.00,4.309653,0.082192,19.934668,0.048162,0.543510
1,2018-09-19,2018-10-19,1,25.00,2.883984,0.082192,19.934668,0.048162,0.543510
2,2018-09-20,2018-10-19,1,25.00,2.929014,0.079452,19.888666,-0.002310,0.541407
3,2018-09-18,2018-10-19,0,25.00,3.784879,0.084932,18.997334,-0.034084,0.613629
4,2018-09-18,2018-10-19,1,25.00,2.790793,0.084932,18.997334,-0.034084,0.613629
...,...,...,...,...,...,...,...,...,...
171330,2023-08-16,2023-09-15,0,750.00,1.157786,0.082192,225.600006,-0.032103,0.430304
171331,2023-08-17,2023-09-15,0,733.33,1.185342,0.079452,219.220001,-0.028688,0.432902
171332,2023-08-16,2023-09-15,0,758.33,1.167400,0.082192,225.600006,-0.032103,0.430304
171333,2023-08-17,2023-09-15,0,750.00,1.205247,0.079452,219.220001,-0.028688,0.432902


In [ ]:
# Select Implied Volatility from ATM options 
# spot price is taken from the close price on a given trade_date
# implied volatility is taken from the option with the closest strike price to the spot price for a given (trade_date, exp_date) pair

# Get the spot price 
spot_price = close_df['Close']
spot_price.columns = ['Spot_price']
spot_price

# Get the implied volatility 
implied_vol = df['implied_volatility']
implied_vol.columns = ['Implied_volatility']
implied_vol

